O modelo tem o intuito de definir pontos criticos de acidentes com base na densidade, para isso é necessario definir o valor da densidade e o valor de numeros minimos

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

In [3]:
df_acidentes = pd.read_csv('Arquivos/df_acidentes_volume.csv', encoding='utf-8', sep=',')

Gerar uma coluna com numeros para gerar o loop no modelo

In [ ]:
def coluna_em_numeros(df):
    concessionaria = {'Fernão Dias': 1,
                      'Litoral Sul': 2,
                      'Régis Bittencourt': 3,
                      'Planalto Sul': 4,
                      'Fluminense': 5}
    df['Concessionaria_1'] = df['Concessionaria'].replace(concessionaria).astype(int)

    rodovia = {'MG-BR381': 1,
               'MG-CONT': 3,
               'SP-BR381': 2,
               'BR101': 1,
               'BR116': 2,
               'BR376': 3,
               'CW-BR116': 1,
               'PR-BR116': 2,
               'SP-BR116': 3,
               'BR 101': 1,
               'BR116/PR': 1,
               'BR116/SC': 2}
    df['Rodovia_1'] = df['Rodovia'].replace(rodovia).astype(int)

    df['Sentido_1'] = df['Sentido'].replace('S', 1).replace('N', 2).astype(int)
    return df

df_acidentes = coluna_em_numeros(df_acidentes)

In [9]:
df_acidentes.drop(columns='Veiculos').head()

,NumOcorrencia,DataOcorrencia,DescrOcorrencia,Sentido,Rodovia,Numveic,Ilesos,VitimasLeves,VitimasModeradas,VitimasGraves,...,Veiculo_Passageiro,Mes_Ano,Praca_pedagio,Volume_Equivalente,Volume_Total,Ano_Marginal,Marginal,Concessionaria_1,Rodovia_1,Sentido_1
0,15,2009-01-01,Acidente com VITIMA,S,BR101,1.0,NaN,1.0,NaN,NaN,...,False,2009-01-01,Praça 04,NaN,NaN,NaN,False,2,1,1
1,31,2009-01-01,Acidente com VITIMA,N,BR101,2.0,2.0,NaN,1.0,NaN,...,False,2009-01-01,Praça 03,NaN,NaN,NaN,False,2,1,2
2,231,2009-01-06,Acidente com VITIMA,S,BR101,1.0,NaN,1.0,NaN,NaN,...,False,2009-01-01,Praça 05,NaN,NaN,2009.0,True,2,1,1
3,150,2009-01-04,Acidente com Danos Materiais,S,BR376,1.0,1.0,NaN,NaN,NaN,...,False,2009-01-01,Praça 02,NaN,NaN,NaN,False,2,3,1
4,36,2009-01-07,Acidente com VITIMA,S,BR101,1.0,NaN,1.0,NaN,NaN,...,False,2009-01-01,Praça 04,NaN,NaN,NaN,False,2,1,1


Como o foco não são pedestres ou ciclistas, foram retirados do modelo

In [10]:
df_acidentes = df_acidentes.query('TipoAcidente != "Atropelamento - Pedestre" and TipoAcidente != "Atropelamento - Ciclista"')

Primeiro avalio os desvios padrão para definir qual vai ser minha densidade, gerei densidades de 50 a 150 metros

In [14]:
valor_minimo = 50
valor_maximo = 150

contagem_linhas = (valor_maximo-valor_minimo)* \
                  (df_acidentes['Concessionaria_1'].nunique())*\
                   (df_acidentes['Sentido_1'].nunique())*\
                   (df_acidentes['Rodovia_1'].nunique())\

arquivo_metricas = np.empty((contagem_linhas, 8), dtype=float)
numero_linha = 0

São tres loops, por concessionaria, por sentido, por rodovia

In [35]:
concessionaria = df_acidentes['Concessionaria_1'].unique()
for i in concessionaria:
    df1 = df_acidentes.query("Concessionaria_1 == @i").copy()
    sentido = df1['Sentido_1'].unique()
    
    for j in sentido:
        df2 = df1.query("Sentido_1 == @j").copy()
        rodovia = df2['Rodovia_1'].unique()
        
        for k in rodovia:
            df3 = df2.query("Rodovia_1 == @k").copy()
            X = df3[['Longitude', 'Latitude']]
            X = X.dropna()
            X = np.array(X)
            X = X.astype(float)
            for m in range(valor_minimo, valor_maximo):
                modelo = DBSCAN(eps=(m/111320), min_samples=5).fit(X)
                class_predictions = modelo.labels_

                df3['CLUSTERS_DBSCAN'] = class_predictions #coluna nova gerada com o numero do cluster
                y_pred = modelo.fit_predict(X)

                labels = modelo.labels_
                n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0) #quantidade de grupos
                n_noise_ = list(labels).count(-1) #ruidos

                #armazenando informação de cada cluster, como rodovia, kmmt inicial e final, contagem de acidentes, etc.
                kmmin = df3.groupby(['Rodovia','CLUSTERS_DBSCAN']) ['kmmt'].min().reset_index(name="kmmin")
                kmmax = df3.groupby(['Rodovia','CLUSTERS_DBSCAN'])['kmmt'].max().reset_index(name="kmmax")
                kmmax.drop(["Rodovia","CLUSTERS_DBSCAN"], axis=1, inplace=True)

                contagem = df3.groupby(['Rodovia','CLUSTERS_DBSCAN']) ['CLUSTERS_DBSCAN'].count().reset_index(name="contagem")
                contagem.drop(["Rodovia","CLUSTERS_DBSCAN"], axis=1, inplace=True)

                somaUPS = df3.groupby(['Rodovia','CLUSTERS_DBSCAN']) ['UPS'].sum().reset_index(name="somaUPS")
                somaUPS.drop(["Rodovia","CLUSTERS_DBSCAN"], axis=1, inplace=True)


                resumo = pd.concat([kmmin, kmmax, contagem, somaUPS], axis=1)
                resumo['extensao'] = resumo['kmmax'] - resumo['kmmin']
                resumo = resumo.query("CLUSTERS_DBSCAN >= 0")
                resumo = resumo.query("extensao > 0")

                #gerando o arquivo com as informações do desvio_padrão, como contagem de cluster, media da extensão, desvio padrão da extensão
                arquivo_metricas[numero_linha][0] = i
                arquivo_metricas[numero_linha][1] = j
                arquivo_metricas[numero_linha][2] = k
                arquivo_metricas[numero_linha][3] = m
                arquivo_metricas[numero_linha][4] = resumo['CLUSTERS_DBSCAN'].count()
                arquivo_metricas[numero_linha][5] = resumo['extensao'].mean().astype(float)
                arquivo_metricas[numero_linha][6] = resumo['extensao'].std().astype(float)
                arquivo_metricas[numero_linha][7] = resumo['extensao'].sum().astype(float)
                numero_linha += 1

resumo_metricas = pd.DataFrame(arquivo_metricas, columns=['Concessionaria', 'Sentido', 'Rodovia', 'Meters', 'N Cluster','Media Extensao', 'Desvio Padrao Extensao', 'Soma Extensao'])

IndexError: index 1800 is out of bounds for axis 0 with size 1800

In [26]:
resumo_metricas = resumo_metricas.query('Meters != 0.0')
resumo_metricas

,Concessionaria,Sentido,Rodovia,Meters,N Cluster,Media Extensao,Desvio Padrao Extensao,Soma Extensao
0,2.0,1.0,1.0,50.0,7.0,0.100000,6.393005e-15,0.7
1,2.0,1.0,1.0,51.0,13.0,0.100000,5.151514e-15,1.3
2,2.0,1.0,1.0,52.0,17.0,0.105882,2.425356e-02,1.8
3,2.0,1.0,1.0,53.0,25.0,0.104000,2.000000e-02,2.6
4,2.0,1.0,1.0,54.0,34.0,0.102941,1.714986e-02,3.5
...,...,...,...,...,...,...,...,...
1595,1.0,2.0,2.0,145.0,69.0,1.115942,1.537150e+00,77.0
1596,1.0,2.0,2.0,146.0,67.0,1.152239,1.589869e+00,77.2
1597,1.0,2.0,2.0,147.0,63.0,1.231746,1.668283e+00,77.6
1598,1.0,2.0,2.0,148.0,62.0,1.253226,1.775767e+00,77.7


In [33]:
import matplotlib.pyplot as plt
colunas = ['Concessionaria', 'Sentido', 'Rodovia']
resumo_metricas[colunas].astype(int)
for i in range (1, resumo_metricas['Concessionaria'].max()+1):
    for j in range (1, resumo_metricas['Sentido'].max()+1):
        for k in range (1, resumo_metricas['Rodovia'].max()+1):
            plt.figure(figsize=(10, 6))
            plt.plot(resumo_metricas['N Cluster'], resumo_metricas['Soma Extensao'], color='blue')
            plt.title("Desvio Padrão da Extensão em relação à Extensão")
            plt.xlabel("Extensão")
            plt.ylabel("Desvio Padrão da Extensão")
            plt.grid(True)
            plt.show()

KeyError: 'N Cluster'

<Figure size 1000x600 with 0 Axes>

Foi escolhido o valor de 100 metros para densidade

In [ ]:
df_acidentes['CssRodSentido'] = df_acidentes.apply(lambda x: '%s.%s.%s' % (x['Concessionaria'], x['Rodovia'], x['Sentido']), axis=1)

resumo_total = pd.DataFrame()
acidentes_total = pd.DataFrame()
ultimo_DBSCAN = 0

concessionaria = df_acidentes['Concessionaria_1'].unique()

for i in concessionaria:
    df0 = df_acidentes.query("Concessionaria_1 == @i").copy()
    sentido = df0['Sentido_1'].unique()
    for j in sentido:
        df1 = df0.query("Sentido_1 == @j").copy()
        rodovia = df1['Rodovia_1'].unique()
        for k in rodovia:
            df2 = df1.query("Rodovia_1 == @k").copy()
            X = df2[['Longitude', 'Latitude']]
            X = X.dropna()
            X = np.array(X)
            X = X.astype(float)

            modelo = DBSCAN(eps=(100/111320), min_samples=5).fit(X)

            class_predictions = modelo.labels_

            df2['CLUSTERS_DBSCAN'] = class_predictions
            y_pred = modelo.fit_predict(X)

            labels = modelo.labels_
            n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)  # quantidade de grupos
            n_noise_ = list(labels).count(-1)  # ruidos

            #armazenando informação de cada cluster, como rodovia, kmmt inicial e final, contagem de acidentes, etc.
            kmmin = df2.groupby(['Concessionaria', 'Sentido', 'Rodovia', 'CssRodSentido', 'CLUSTERS_DBSCAN'])['kmmt'].min().reset_index(name="kmmin")
            kmmin['CLUSTERS_DBSCAN'] = kmmin['CLUSTERS_DBSCAN'].apply(lambda x: x + ultimo_DBSCAN if x >= 0 else x)

            kmmax = df2.groupby(['CssRodSentido', 'CLUSTERS_DBSCAN'])['kmmt'].max().reset_index(name="kmmax")
            kmmax.drop(["CssRodSentido", "CLUSTERS_DBSCAN"], axis=1, inplace=True)

            contagem = df2.groupby(['CssRodSentido', 'CLUSTERS_DBSCAN'])['CLUSTERS_DBSCAN'].count().reset_index(name="contagem")
            contagem.drop(["CssRodSentido", "CLUSTERS_DBSCAN"], axis=1, inplace=True)


            somaUPS = df2.groupby(['CssRodSentido', 'CLUSTERS_DBSCAN'])['UPS'].sum().reset_index(name="somaUPS")
            somaUPS.drop(["CssRodSentido", "CLUSTERS_DBSCAN"], axis=1, inplace=True)

            resumo = pd.concat([kmmin, kmmax, contagem, somaUPS], axis=1)
            resumo['extensao'] = resumo['kmmax'] - resumo['kmmin']
            
            #excluindo os ruidos
            resumo = resumo.query("CLUSTERS_DBSCAN >= 0")

            #devolvendo para o df_acidentes qual o numero do cluster que o acidente pertence, senão retorna vazio
            df2['CLUSTERS_DBSCAN'] = df2.apply(lambda row: resumo.loc[(resumo.CssRodSentido == row['CssRodSentido']) & (row['kmmt'] >= resumo.kmmin) &(row['kmmt'] < resumo.kmmax), 'CLUSTERS_DBSCAN'].values[0] if not resumo.loc[(resumo.CssRodSentido == row['CssRodSentido']) & (row['kmmt'] >= resumo.kmmin) & (row['kmmt'] < resumo.kmmax), 'CLUSTERS_DBSCAN'].empty else None, axis=1)

            df2.drop(["Sentido_1", "Rodovia_1", "Concessionaria_1"], axis=1, inplace=True)
            df2.reset_index(inplace=True)

            acidentes_total = pd.concat([acidentes_total, df2], axis=0, ignore_index=True)
            resumo_total = pd.concat([resumo_total, resumo], axis=0, ignore_index=True)
            
            #pego o ultimo valor do cluster nesse loop e utilizo como primeiro numero do proximo para não duplicar valor de cluster
            ultimo_DBSCAN = resumo_total['CLUSTERS_DBSCAN'].max() +1


In [ ]:
acidentes_total['Cluster'] = np.where(acidentes_total['CLUSTERS_DBSCAN'].notna(), True, False)
resumo_total.to_csv('Arquivos/Cluster/resumo_acidentes_volume.csv', encoding='latin1', index=False)
acidentes_total.to_csv('Arquivos/Cluster/df_acidentes_volume.csv', encoding='latin1', index=False)


Para gerar um modelo que comparasse os clusters no decorrer dos anos, foi definido que os dados seriam separados em dois, entre os anos de 2011-2016 e entre 2017-2022

In [ ]:
def cluster_comparativo(df_base, df_ano_1, df_ano_2):

    df_base['CssRodSentido'] = df_base.apply(lambda x: '%s.%s.%s' % (x['Concessionaria'], x['Rodovia'], x['Sentido']), axis=1)
    concessionaria = df_base['Concessionaria_1'].unique()
    resumo_total = pd.DataFrame()
    df_total = pd.DataFrame()
    ultimo_DBSCAN = 0
    for i in concessionaria:
        df0 = df_base.query("Concessionaria_1 == @i").copy()
        sentido = df0['Sentido_1'].unique()
        for j in sentido:
            df1 = df0.query("Sentido_1 == @j").copy()
            rodovia = df1['Rodovia_1'].unique()
            for k in rodovia:
                df2 = df1.query("Rodovia_1 == @k").copy()
                X = df2[['Longitude', 'Latitude']]
                X = X.dropna()
                X = np.array(X)
                X = X.astype(float)

                modelo = DBSCAN(eps=100/111320, min_samples=5).fit(X)

                class_predictions = modelo.labels_

                df2['CLUSTERS_DBSCAN'] = class_predictions
                y_pred = modelo.fit_predict(X)

                labels = modelo.labels_
                n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)  # quantidade de grupos
                n_noise_ = list(labels).count(-1)  # ruidos

                kmmin = df2.groupby(['Concessionaria', 'Sentido', 'Rodovia', 'CssRodSentido', 'CLUSTERS_DBSCAN'])[
                    'kmmt'].min().reset_index(name="kmmin")
                kmmin['CLUSTERS_DBSCAN'] = kmmin['CLUSTERS_DBSCAN'].apply(lambda x: x + ultimo_DBSCAN if x >= 0 else x)

                kmmax = df2.groupby(['CssRodSentido', 'CLUSTERS_DBSCAN'])['kmmt'].max().reset_index(name="kmmax")
                kmmax.drop(["CssRodSentido", "CLUSTERS_DBSCAN"], axis=1, inplace=True)

                contagem = df2.groupby(['CssRodSentido', 'CLUSTERS_DBSCAN'])['CLUSTERS_DBSCAN'].count().reset_index(
                    name="contagem")
                contagem.drop(["CssRodSentido", "CLUSTERS_DBSCAN"], axis=1, inplace=True)

                somaUPS = df2.groupby(['CssRodSentido', 'CLUSTERS_DBSCAN'])['UPS'].sum().reset_index(name="somaUPS")
                somaUPS.drop(["CssRodSentido", "CLUSTERS_DBSCAN"], axis=1, inplace=True)

                resumo = pd.concat([kmmin, kmmax, contagem, somaUPS], axis=1)
                resumo['extensao'] = resumo['kmmax'] - resumo['kmmin']

                resumo = resumo.query("CLUSTERS_DBSCAN >= 0")

                resumo['UPS_Ano_1'] = resumo.apply(
                    lambda row: df_ano_1[(df_ano_1.CssRodSentido == row['CssRodSentido']) &
                                         (df_ano_1.kmmt >= row['kmmin']) &
                                         (df_ano_1.kmmt < row['kmmax'])].UPS.sum(), axis=1)

                resumo['UPS_Ano_2'] = resumo.apply(
                    lambda row: df_ano_2[(df_ano_2.CssRodSentido == row['CssRodSentido']) &
                                         (df_ano_2.kmmt >= row['kmmin']) &
                                         (df_ano_2.kmmt < row['kmmax'])].UPS.sum(), axis=1)

                df2['CLUSTERS_DBSCAN'] = df2.apply(
                    lambda row: resumo.loc[(resumo.CssRodSentido == row['CssRodSentido']) &
                                           (row['kmmt'] >= resumo.kmmin) &
                                           (row['kmmt'] < resumo.kmmax), 'CLUSTERS_DBSCAN'].values[0] if not resumo.loc[
                        (resumo.CssRodSentido == row['CssRodSentido']) &
                        (row['kmmt'] >= resumo.kmmin) &
                        (row['kmmt'] < resumo.kmmax), 'CLUSTERS_DBSCAN'].empty else None, axis=1)
                
                #gerando para o ano 1
                df_ano_1['CLUSTERS_DBSCAN'] = df_ano_1.apply(
                    lambda row: resumo.loc[(resumo.CssRodSentido == row['CssRodSentido']) &
                                           (row['kmmt'] >= resumo.kmmin) &
                                           (row['kmmt'] < resumo.kmmax), 'CLUSTERS_DBSCAN'].values[0] if not resumo.loc[
                        (resumo.CssRodSentido == row['CssRodSentido']) &
                        (row['kmmt'] >= resumo.kmmin) & (
                                    row['kmmt'] < resumo.kmmax), 'CLUSTERS_DBSCAN'].empty else None, axis=1)

                #gerando para o ano 2
                df_ano_2['CLUSTERS_DBSCAN'] = df_ano_2.apply(
                    lambda row: resumo.loc[(resumo.CssRodSentido == row['CssRodSentido']) &
                                           (row['kmmt'] >= resumo.kmmin) &
                                           (row['kmmt'] < resumo.kmmax), 'CLUSTERS_DBSCAN'].values[0] if not resumo.loc[
                        (resumo.CssRodSentido == row['CssRodSentido']) &
                        (row['kmmt'] >= resumo.kmmin) & (
                                    row['kmmt'] < resumo.kmmax), 'CLUSTERS_DBSCAN'].empty else None, axis=1)

                df_total = pd.concat([df_total, df2, df_ano_1, df_ano_2], axis=0, ignore_index=True)
                df_total.drop(["CssRodSentido", "Sentido_1", "Rodovia_1", "Concessionaria_1"], axis=1, inplace=True)

                resumo_total = pd.concat([resumo_total, resumo], axis=0, ignore_index=True)
                ultimo_DBSCAN = resumo_total['CLUSTERS_DBSCAN'].max()

    return resumo_total, df_total

resumo_total, df_total = cluster_comparativo(df_acidentes, 2011, 2017)